In [1]:
import torch
import torch.nn as nn
from torch import optim

import numpy as np
import pandas as pd

from utils import FaceKeypointDataset, transform, train_test_split, train
from model.network import Net

In [2]:
annotations_df = pd.read_csv('data/Annotations/annotations.csv')
images_dir = 'data/Images/'

In [3]:
dataset = FaceKeypointDataset(annotations=annotations_df, 
                              root_dir=images_dir, 
                              transform=transform)

In [4]:
train_loader, val_loader, test_loader = train_test_split(dataset, 
                                                         train_size=0.8, 
                                                         val_size=0.1, 
                                                         batch_size=64)

In [5]:
model = Net()

In [6]:
criterion_keypoints = nn.MSELoss()
criterion_bbox = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [7]:
train(model, 
      criterion_keypoints, 
      criterion_bbox, 
      optimizer, 
      train_loader, 
      val_loader, 
      epochs=50, 
      device='cuda', 
      bbox_weight=1.0)

Epoch 1/50 - Training:   9%|▊         | 18/206 [00:22<04:09,  1.33s/it, Batch Loss=1.85e+5]

Skipping 4566 due to error: image file is truncated (6 bytes not processed)


Epoch 1/50 - Training:  30%|██▉       | 61/206 [01:20<03:03,  1.27s/it, Batch Loss=8.48e+4]

Skipping 16413 due to error: [Errno 2] No such file or directory: 'C:\\Users\\first\\Desktop\\GithubForSchool\\face-keypoint-detection\\data\\Images\\image32677.jpg'


Epoch 1/50 - Training:  33%|███▎      | 69/206 [01:30<02:55,  1.28s/it, Batch Loss=2.12e+5]

...

Epoch 2/50 - Training:  16%|█▌        | 32/206 [00:39<04:44,  1.64s/it, Batch Loss=5.66e+5]

Skipping 16417 due to error: [Errno 2] No such file or directory: 'C:\\Users\\first\\Desktop\\GithubForSchool\\face-keypoint-detection\\data\\Images\\image59010.jpg'


Epoch 2/50 - Training:  17%|█▋        | 36/206 [00:43<03:29,  1.23s/it, Batch Loss=1.41e+5]

Skipping 16411 due to error: [Errno 2] No such file or directory: 'C:\\Users\\first\\Desktop\\GithubForSchool\\face-keypoint-detection\\data\\Images\\image19281.jpg'


KeyboardInterrupt: 